# InSituPy demonstration - Add annotations

In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from insitupy import read_xenium

## Previous steps

1. Download the example data for demonstration: [01_InSituPy_demo_download_data.ipynb](./01_InSituPy_demo_download_data.ipynb)
2. Register images from external stainings: [02_InSituPy_demo_register_images.ipynb](./02_InSituPy_demo_register_images.ipynb)
3. Visualize data with napari and do preprocessing steps: [03_InSituPy_demo_analyze.ipynb](./03_InSituPy_demo_analyze.ipynb)

At this point, the structure of the data should look like this:

    ```
    ./demo_dataset
    ├───cropped_processed
    ├───output-XETG00000__slide_id__sample_id
    │   ├───analysis
    │   │   ├───clustering
    │   │   ├───diffexp
    │   │   ├───pca
    │   │   ├───tsne
    │   │   └───umap
    │   └───cell_feature_matrix
    ├───registered_images
    ├───registration_qc
    └───unregistered_images
    ```


## Load Xenium data into `XeniumData` object

Now the Xenium data can be parsed by providing the data path to `XeniumData`

In [3]:
insitupy_project = Path("demo_dataset/demo_insitupy_project")

In [4]:
xd = read_xenium(insitupy_project)

In [5]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy

In [6]:
xd.load_images()
xd.load_cells()

Loading images...
Loading cells...
	No alternative cells found...


Note: That the `annotations` and `regions` modalities are not found here is expected. Annotations and regions are added in a later step.

In [7]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-TestKey', 'annotation-demo', 'annotation-demo2', 'annotation-demo3'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear

## Create annotations

For the analysis of spatial transcriptomic datasets the inclusion of annotations from experts of disease pathology is key. Here, we demonstrate two ways how to annotate Xenium data
1. Within `InSituPy` using the `napari` viewer.
2. In [QuPath](https://qupath.github.io/).

Importantly, `InSituPy` differentiates between "regions" and histological "annotations". Annotations consist of polygons with each polygon getting assigned to a certain “class” (e.g. “tumor cells”, “immune cells”, "stroma", etc.) and a “key” (e.g. the name of the pathologist doing the annotations). The classes within one key do not have to be unique (multiple annotations could contain tumor cells) and a unique identifier is used to differentiate between the polygons. Regions also consist of polygons and a key for a cohesive group of polygons but each polygon within one key is required to have a unique name. Regions can delineate the positions of TMA cores or the position of different tissue sections or regions of interests within the same dataset. 

### 1. In `napari` viewer

First visualize the Xenium data using `.show()`.

In [8]:
xd.show()

Invalid schema for package 'ome-types', please run 'npe2 validate ome-types' to check for manifest errors.


A new annotation layer can be added using the "Add geometries" widget on the bottom right.
<center><img src="./demo_screenshots/add_geometries_widget.JPG" width="200"/></center>

As described above, `InSituPy` differentiates between "regions" and histological "annotations". Since napari creates separate layers for point and shape annotations, the "annotations" are split further into two subtypes, resulting in a total of three possible geometry types one can chose from:
1. Geometric annotations <br>
<img src="./demo_screenshots/annotation_layer.jpg" width="200"/>
2. Point annotations <br>
<img src="./demo_screenshots/point_layer.jpg" width="200"/>
3. Region <br>
<img src="./demo_screenshots/region_layer.jpg" width="200"/>

Since `InSituPy` uses different icons to differentiate between the types, it is important to add the geometries via this widget and not via the normal `napari` annotation panel.

After adding the geometries, they can be imported into the `InSituPy` object using `.store_geometries()`:

In [9]:
xd.store_geometries()

Added 3 new annotations to key 'TestKey'


In [10]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-TestKey', 'annotation-demo', 'annotation-demo2', 'annotation-demo3'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       TestKey:	3 annotations, 1 class ('TestClass') 



### 2. Create annotations in QuPath

To create annotations in QuPath, follow these steps:

1. Select a annotation tool from the bar on the top left:

<center><img src="./demo_screenshots/qupath_annotation_buttons.jpg" width="300"/></center>

2. Add as many annotations as you want and label them by setting classes in the annotation list. Do not forget to press the "Set class" button:

<center><img src="./demo_screenshots/qupath_annotation_list.jpg" width="350"/></center>

3. Export annotations using `File > Export objects as GeoJSON`. Tick `Pretty JSON` to get an easily readable JSON file. The file name needs to have following structure: `annotation-{slide_id}__{sample_id}__{annotation_label}`.

### Import annotations into `XeniumData`

For demonstration purposes, we created dummy annotation files in `./demo_annotations/`. To add the annotations to `XeniumData` follow the steps below.



## Import annotations and regions

In [11]:
xd.import_annotations(
    files=[
        "./demo_annotations/annotations-0001879__Replicate 1__demo.geojson",
        "./demo_annotations/annotations-0001879__Replicate 1__demo2.geojson",
        "./demo_annotations/annotations-mixed_types.geojson"
           ],
    keys=["demo", "demo2", "demo3"]
    )

Importing annotations...


In [12]:
xd.annotations.demo

,objectType,geometry,name,color,origin,scale,layer_type
id,,,,,,,
bd3aacca-1716-4df8-91dd-bf8f6413a7bd,annotation,"POLYGON ((8863.00000 10814.00000, 8863.00000 1...",Positive,"[250, 62, 62]",file,"(0.2125, 0.2125)",Shapes
69814505-4059-42cd-8df2-752f7eb0810d,annotation,"POLYGON ((13096.00000 12492.00000, 13072.40000...",Positive,"[250, 62, 62]",file,"(0.2125, 0.2125)",Shapes
1957cd32-0a21-4b45-9dae-ecf236217140,annotation,"POLYGON ((30975.26000 22938.00000, 30982.00000...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes
19d2197a-1b8e-456f-8223-fba74641ac1c,annotation,"POLYGON ((31165.00000 16408.00000, 31149.00000...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes


In [13]:
xd.annotations.demo2

,objectType,geometry,name,color,origin,scale,layer_type
id,,,,,,,
1970eccb-ad38-4b4b-b7a8-54509027b57d,annotation,"POLYGON ((25319.00000 3892.00000, 25313.00000 ...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes
a3b32cce-1bb9-4a6f-b1d1-9e0c44420cfa,annotation,"POLYGON ((30950.00000 10855.00000, 30944.00000...",Positive,"[250, 62, 62]",file,"(0.2125, 0.2125)",Shapes
92bfe928-a21f-4864-b7cb-f0d300113d88,annotation,"MULTIPOLYGON (((21534.00000 19541.00000, 21534...",Other,"[255, 200, 0]",file,"(0.2125, 0.2125)",Shapes
a6c17a54-6839-40b2-8531-c9227635f344,annotation,"POLYGON ((6501.00000 17126.00000, 6495.00000 1...",Other,"[255, 200, 0]",file,"(0.2125, 0.2125)",Shapes
e78efe2f-d185-4ab6-9cc9-6621897f3662,annotation,"POLYGON ((29519.63000 18523.00000, 29476.00000...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes


In [14]:
xd.annotations.demo3

,objectType,geometry,name,color,origin,scale,layer_type
id,,,,,,,
8f57c3c3-2216-48b7-99bd-aba12d8c3c41,annotation,"POLYGON ((18016.00000 10643.50000, 18013.24000...",Stroma,"[150, 200, 150]",file,"(0.2125, 0.2125)",Shapes
7e8f8db4-81d4-472e-8e93-0fc756df87aa,annotation,"POLYGON ((12322.00000 6758.00000, 12320.34000 ...",Stroma,"[150, 200, 150]",file,"(0.2125, 0.2125)",Shapes
38a48ddb-f33c-4c61-b996-330b25d84081,annotation,"LINESTRING (16943.80000 7758.84000, 18249.5800...",Necrosis,"[50, 50, 50]",file,"(0.2125, 0.2125)",Shapes
eee244c9-e919-41ae-bb91-44c7abcc0cec,annotation,"LINESTRING (11687.64000 6279.51000, 13439.6900...",Immune cells,"[160, 90, 160]",file,"(0.2125, 0.2125)",Shapes
e3d4c0b6-0998-4692-ab7d-f580f713e275,annotation,POINT (23982.29000 7682.65000),unclassified,"[0, 0, 0]",file,"(0.2125, 0.2125)",Points
e9105240-3b35-489e-994f-e8f9c4786516,annotation,"MULTIPOINT (19857.20000 7768.09000, 20056.2200...",Stroma,"[150, 200, 150]",file,"(0.2125, 0.2125)",Points
2802df97-78ad-44ac-8e6b-d9b9406c8e3f,annotation,"MULTIPOINT (15871.96000 9437.25000, 16611.9600...",Tumor,"[200, 0, 0]",file,"(0.2125, 0.2125)",Points


## Load regions

Regions can be created in QuPath either as described above or using tools like the TMA dearrayer. They are also exported as objects as annotations but different to annotations they do not have a classification and each name of a region has to be unique.

In the following demo regions are read. One of the region files has non-unique names to demonstrate the warning that appears in this case.

### Classes have to be unique in regions

When reading an "Annotation" `.geojson` as shown below, the `import_regions` function throws an error indicating that in regions only one geometry per class is allowed. Further, only normal polygons (`shapely.Polygon`-typed) are allowed. Any other types of geometries (Points, Lines, MultiPolygons, ...) are skipped.

In [15]:
xd.import_regions(
    files=[
        "./demo_annotations/annotations-mixed_types.geojson"
        ],
    keys=['test'])

Importing regions...


C:\Users\ge37voy\Github\InSituPy\insitupy\_core\dataclasses.py:122: UserWarning: The names of the regions for key 'test' were not unique and thus the key was skipped. In regions only one geometry per class is allowed.
  warnings.warn(
C:\Users\ge37voy\Github\InSituPy\insitupy\_core\insitudata.py:729: UserWarning: Some regions were not pure Polygon objects and skipped.
  self.regions.add_data(data=file,


Multiple regions can be imported simultaneously.

In [16]:
xd.import_regions(
    files=[
        "./demo_regions/regions-0001879__Replicate 1__demo_regions.geojson",
        "./demo_regions/regions-0001879__Replicate 1__TMA.geojson",
        ],
    keys=['demo_regions', 'TMA'])

Importing regions...


Properties of the `anotations` and `regions` modalities can be inspected in the InSituData representation:

In [17]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-TestKey', 'annotation-demo', 'annotation-demo2', 'annotation-demo3'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       TestKey:	3 annotations, 1 class ('TestClass') 
       demo:	4 annotations, 2 classes ('Positive','Negative') 
       demo2:	5 annotations, 3 

## Visualization of `annotations` and `regions` using `napari` viewer

Ìf the `InSituData` object only contains `.annotations` or `.regions` attributes, one can choose between the "Add geometries" and "Show geometries" widgets:
<center><img src="./demo_screenshots/toggle_geometry_widgets.jpg" width="200"/></center>

Annotations and regions stored in the `InSituData` object can be visualized using the "Show geometries" widget:
<center><img src="./demo_screenshots/show_geometries_widget.jpg" width="200"/></center>

To show the names of the annotations, tick "Show names":
<center><img src="./demo_screenshots/show_names_example.jpg" width="200"/></center>


In [18]:
xd.show()

### Assign annotations to observations

To use the annotations in analyses (e.g. to select only observations within a certain annotation or compare gene expression between different annotations) one can use the `assign_annotations` function. It adds columns containing the annotation class to `xd.matrix.obs`. The column has the syntax `annotation-{Label}` and if an observation is not part of any annotation within this label, it contains `NaN`. 

In [19]:
xd.assign_annotations(overwrite=True)

Assigning key 'TestKey'...
Existing column "annotation-TestKey" is overwritten.
Assigning key 'demo'...
Existing column "annotation-demo" is overwritten.
Assigning key 'demo2'...
Existing column "annotation-demo2" is overwritten.
Assigning key 'demo3'...
Existing column "annotation-demo3" is overwritten.


In [20]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-TestKey', 'annotation-demo', 'annotation-demo2', 'annotation-demo3'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       TestKey:	3 annotations, 1 class ('TestClass') ✔
       demo:	4 annotations, 2 classes ('Positive','Negative') ✔
       demo2:	5 annotations, 

After assigning the annotations, the labels analyzed here are marked with a ✔:

In [21]:
xd.regions.demo_regions

,objectType,name,geometry,origin,scale,layer_type
id,,,,,,
2d0da635-c408-459f-9178-839097fe5a98,annotation,Region1,"POLYGON ((7362.00000 6221.00000, 10672.00000 6...",file,"(0.2125, 0.2125)",Shapes
ce6c2342-620d-4f44-be03-68a4454e9b33,annotation,Region2,"POLYGON ((21373.00000 6383.00000, 26418.00000 ...",file,"(0.2125, 0.2125)",Shapes
70a125ec-c53e-469b-8927-efe224e504c1,annotation,Region3,"POLYGON ((9933.00000 12745.00000, 15942.00000 ...",file,"(0.2125, 0.2125)",Shapes


Following cells show examples how to explore the assigned annotations:

In [22]:
# print number of cells within one annotation
xd.cells.matrix.obs["annotation-demo2"].notna().sum()

9431

In [23]:
xd.annotations.demo2

,objectType,geometry,name,color,origin,scale,layer_type
id,,,,,,,
1970eccb-ad38-4b4b-b7a8-54509027b57d,annotation,"POLYGON ((25319.00000 3892.00000, 25313.00000 ...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes
a3b32cce-1bb9-4a6f-b1d1-9e0c44420cfa,annotation,"POLYGON ((30950.00000 10855.00000, 30944.00000...",Positive,"[250, 62, 62]",file,"(0.2125, 0.2125)",Shapes
92bfe928-a21f-4864-b7cb-f0d300113d88,annotation,"MULTIPOLYGON (((21534.00000 19541.00000, 21534...",Other,"[255, 200, 0]",file,"(0.2125, 0.2125)",Shapes
a6c17a54-6839-40b2-8531-c9227635f344,annotation,"POLYGON ((6501.00000 17126.00000, 6495.00000 1...",Other,"[255, 200, 0]",file,"(0.2125, 0.2125)",Shapes
e78efe2f-d185-4ab6-9cc9-6621897f3662,annotation,"POLYGON ((29519.63000 18523.00000, 29476.00000...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes


In [24]:
xd.cells.matrix.obs["annotation-demo2"][xd.cells.matrix.obs["annotation-demo2"].notna()]

4921         Other
4922         Other
4923         Other
4924         Other
4925         Other
            ...   
165374    Negative
165375    Negative
165376    Negative
165377    Negative
165378    Negative
Name: annotation-demo2, Length: 9431, dtype: object

## Save imported annotations in `InSituPy` project

In [27]:
xd.save()

Updating project in c:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
	Updating cells...
	Updating annotations...
	Updating regions...
Saved.
Reloading following modalities: annotations,cells,images,regions
Loading annotations...
Loading cells...
	No alternative cells found...
Loading images...
Loading regions...


In [33]:
xd.reload()

Reloading following modalities: annotations,cells,images,regions
Loading annotations...
Scale inferred from file.
Scale inferred from file.
Scale inferred from file.
Scale inferred from file.
Loading cells...
	No alternative cells found...
Loading images...
Loading regions...
Scale inferred from file.
Scale inferred from file.


In [34]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-TestKey', 'annotation-demo', 'annotation-demo2', 'annotation-demo3'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       TestKey:	3 annotations, 1 class ('TestClass') ✔
       demo:	4 annotations, 2 classes ('Positive','Negative') ✔
       demo2:	5 annotations, 

In [36]:
xd.show()

In [31]:
from insitupy.io import parse_geopandas

In [32]:
parse_geopandas(r"C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project\regions\240826-161046413312-1a1dc42f\demo_regions.geojson")

,objectType,origin,layer_type,geometry,name,scale
id,,,,,,
2d0da635-c408-459f-9178-839097fe5a98,annotation,file,Shapes,"POLYGON ((7362.00000 6221.00000, 10672.00000 6...",Region1,"[0.2125, 0.2125]"
ce6c2342-620d-4f44-be03-68a4454e9b33,annotation,file,Shapes,"POLYGON ((21373.00000 6383.00000, 26418.00000 ...",Region2,"[0.2125, 0.2125]"
70a125ec-c53e-469b-8927-efe224e504c1,annotation,file,Shapes,"POLYGON ((9933.00000 12745.00000, 15942.00000 ...",Region3,"[0.2125, 0.2125]"


In [30]:
xd.annotations.demo

,objectType,origin,layer_type,geometry,name,color,scale
id,,,,,,,
bd3aacca-1716-4df8-91dd-bf8f6413a7bd,annotation,file,Shapes,"POLYGON ((8863.00000 10814.00000, 8863.00000 1...",Positive,"[250, 62, 62]","(1, 1)"
69814505-4059-42cd-8df2-752f7eb0810d,annotation,file,Shapes,"POLYGON ((13096.00000 12492.00000, 13072.40000...",Positive,"[250, 62, 62]","(1, 1)"
1957cd32-0a21-4b45-9dae-ecf236217140,annotation,file,Shapes,"POLYGON ((30975.26000 22938.00000, 30982.00000...",Negative,"[112, 112, 225]","(1, 1)"
19d2197a-1b8e-456f-8223-fba74641ac1c,annotation,file,Shapes,"POLYGON ((31165.00000 16408.00000, 31149.00000...",Negative,"[112, 112, 225]","(1, 1)"


In [29]:
xd.regions.demo_regions

,objectType,origin,layer_type,geometry,name,scale
id,,,,,,
2d0da635-c408-459f-9178-839097fe5a98,annotation,file,Shapes,"POLYGON ((7362.00000 6221.00000, 10672.00000 6...",Region1,"(1, 1)"
ce6c2342-620d-4f44-be03-68a4454e9b33,annotation,file,Shapes,"POLYGON ((21373.00000 6383.00000, 26418.00000 ...",Region2,"(1, 1)"
70a125ec-c53e-469b-8927-efe224e504c1,annotation,file,Shapes,"POLYGON ((9933.00000 12745.00000, 15942.00000 ...",Region3,"(1, 1)"


In [30]:
xd.show()

In [32]:
xd.annotations.demo

,objectType,origin,layer_type,geometry,name,color,scale
id,,,,,,,
bd3aacca-1716-4df8-91dd-bf8f6413a7bd,annotation,file,Shapes,"POLYGON ((8863.00000 10814.00000, 8863.00000 1...",Positive,"[250, 62, 62]","(1, 1)"
69814505-4059-42cd-8df2-752f7eb0810d,annotation,file,Shapes,"POLYGON ((13096.00000 12492.00000, 13072.40000...",Positive,"[250, 62, 62]","(1, 1)"
1957cd32-0a21-4b45-9dae-ecf236217140,annotation,file,Shapes,"POLYGON ((30975.26000 22938.00000, 30982.00000...",Negative,"[112, 112, 225]","(1, 1)"
19d2197a-1b8e-456f-8223-fba74641ac1c,annotation,file,Shapes,"POLYGON ((31165.00000 16408.00000, 31149.00000...",Negative,"[112, 112, 225]","(1, 1)"


In [123]:
xd.reload()

Reloading following modalities: annotations,cells,images,regions
Loading annotations...


[autoreload of insitupy.io.geo failed: Traceback (most recent call last):
  File "c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
  File "c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "C:\Users\ge37voy\Github\InSituPy\insitupy\io\geo.py", line 10, in <module>
    from ..utils.utils import convert_list_like_entries_to_tuple
ImportError: cannot import name 'convert_list_like_entries_to_tuple'

Loading cells...
	No alternative cells found...
Loading images...
Loading regions...


In [124]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-TestKey', 'annotation-demo', 'annotation-demo2', 'annotation-demo3'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       TestKey:	3 annotations, 1 class ('TestClass') ✔
       demo:	4 annotations, 2 classes ('Positive','Negative') ✔
       demo2:	5 annotations, 

In [125]:
xd.show()

In [ ]:
xd.annotations.demo

,objectType,geometry,name,color,origin,scale,layer_type
id,,,,,,,
bd3aacca-1716-4df8-91dd-bf8f6413a7bd,annotation,"POLYGON ((8863.00000 10814.00000, 8863.00000 1...",Positive,"[250, 62, 62]",file,"(0.2125, 0.2125)",Shapes
69814505-4059-42cd-8df2-752f7eb0810d,annotation,"POLYGON ((13096.00000 12492.00000, 13072.40000...",Positive,"[250, 62, 62]",file,"(0.2125, 0.2125)",Shapes
1957cd32-0a21-4b45-9dae-ecf236217140,annotation,"POLYGON ((30975.26000 22938.00000, 30982.00000...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes
19d2197a-1b8e-456f-8223-fba74641ac1c,annotation,"POLYGON ((31165.00000 16408.00000, 31149.00000...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes


In [126]:
xd.annotations.demo

,objectType,origin,layer_type,geometry,name,color,scale
id,,,,,,,
bd3aacca-1716-4df8-91dd-bf8f6413a7bd,annotation,file,Shapes,"POLYGON ((8863.00000 10814.00000, 8863.00000 1...",Positive,"[250, 62, 62]","(1, 1)"
69814505-4059-42cd-8df2-752f7eb0810d,annotation,file,Shapes,"POLYGON ((13096.00000 12492.00000, 13072.40000...",Positive,"[250, 62, 62]","(1, 1)"
1957cd32-0a21-4b45-9dae-ecf236217140,annotation,file,Shapes,"POLYGON ((30975.26000 22938.00000, 30982.00000...",Negative,"[112, 112, 225]","(1, 1)"
19d2197a-1b8e-456f-8223-fba74641ac1c,annotation,file,Shapes,"POLYGON ((31165.00000 16408.00000, 31149.00000...",Negative,"[112, 112, 225]","(1, 1)"
